# Synthea Case Study
*Lab 26 April 2022*

* https://synthea.mitre.org/
* https://github.com/synthetichealth/synthea

In [91]:
%defaultDatasource jdbc:h2:mem:db

In [92]:
DROP TABLE IF EXISTS Patients;

CREATE TABLE Patients(
    id VARCHAR(36),
    birthdate DATE,
    deathdate DATE,
    ssn VARCHAR(11),
    drivers VARCHAR(9),
    passport VARCHAR(15),
    prefix VARCHAR(4),
    first VARCHAR(20),
    last VARCHAR(20),
    suffix VARCHAR(5),
    maiden VARCHAR(20),
    marital CHAR,
    race VARCHAR(10),
    ethnicity VARCHAR(15),
    gender CHAR,
    birthplace VARCHAR(100),
    address VARCHAR(50),
    city VARCHAR(30),
    state VARCHAR(30),
    county VARCHAR(50),
    zip VARCHAR(10),
    lat DECIMAL(18,15),
    lon DECIMAL(18,15),
    healthcare_expenses VARCHAR(20),
    healthcare_coverage VARCHAR(20),
    PRIMARY KEY(id)
) AS SELECT id,birthdate,deathdate,ssn,drivers,passport,prefix,first,last,suffix,maiden,marital,race,ethnicity,gender,
            birthplace,address,city,state,county,zip,lat,lon,healthcare_expenses,healthcare_coverage
FROM CSVREAD('../../data/synthea/scenario01/csv/patients.csv');

DROP TABLE IF EXISTS Encounters;

CREATE TABLE Encounters(
    id VARCHAR(36),
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    organization VARCHAR(36),
    provider VARCHAR(36),
    payer  VARCHAR(36),
    encounterclass VARCHAR(20),
    code VARCHAR(15),
    description VARCHAR(100),
    base_encounter_cost DECIMAL(8,2),
    total_claim_cost DECIMAL(8,2),
    payer_coverage DECIMAL(8,2),
    reasoncode VARCHAR(15),
    reasondescription VARCHAR(100),
    PRIMARY KEY(id)
) AS SELECT id,start,stop,patient,organization,provider,payer,encounterclass,code,description,base_encounter_cost,
            total_claim_cost,payer_coverage,reasoncode,reasondescription
FROM CSVREAD('../../data/synthea/scenario01/csv/encounters.csv');

DROP TABLE IF EXISTS Conditions;

CREATE TABLE Conditions(
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    encounter VARCHAR(36),
    code VARCHAR(20),
    description_condition VARCHAR(100),
) AS SELECT start,stop,patient,encounter,code,description
FROM CSVREAD('../../data/synthea/scenario01/csv/conditions.csv');

CREATE TABLE Allergies(
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    encounter VARCHAR(36),
    code VARCHAR(20),
    description_condition VARCHAR(100),
    type VARCHAR(20),
    category VARCHAR(20),
    reaction1 VARCHAR(20),
    description1 VARCHAR(100),
    severity1 VARCHAR(20),
    reaction2 VARCHAR(20),
    description2 VARCHAR(100),
    severity2 VARCHAR(20),
) AS SELECT start,stop,patient,encounter,code,description, type,category,reaction1,description1,
    severity1, reaction2, description2, severity2
FROM CSVREAD('../../data/synthea/scenario01/csv/allergies.csv');

org.h2.jdbc.JdbcSQLException:  Cannot drop "ENCOUNTERS" because "EMERGENCY" depends on it; SQL statement

## Listing Patients

In [93]:
SELECT id, to_char(birthdate,'dd-mm-yyyy') birth, to_char(deathdate,'dd-mm-yyyy') death,
       deathdate-birthdate days_life
FROM Patients
LIMIT 7;

## Number of Dead Patients

In [94]:
SELECT COUNT(*) FROM Patients WHERE deathdate IS NOT NULL;

174

## Listing Encounters

In [95]:
SELECT id, to_char(start,'dd-mm-yyyy') start, encounterclass, description
FROM Encounters
LIMIT 7;

# Encounters in the Emergency

## Number of Patients that Visited the Emergency

In [96]:
SELECT COUNT(DISTINCT patient) FROM Encounters
       WHERE encounterclass = 'emergency';

800

In [97]:
SELECT count(DISTINCT patient) FROM Encounters
       WHERE encounterclass = 'emergency' AND
       ( description='Stroke');

39

## Last Encounter in the Emergency

In [98]:
DROP VIEW IF EXISTS Emergency;

CREATE VIEW Emergency AS
SELECT patient, MAX(start) last_encounter
       FROM Encounters
       WHERE encounterclass = 'emergency'
       GROUP BY patient;

## Conditions in the Last Encounter

In [99]:
SELECT e.patient, c.description_condition
       FROM Emergency e, Conditions c
       WHERE e.patient = c.patient
       LIMIT 15;

## Patients that had a Cardiac Arrest or Myocardial Infarction in the Last Encounter

In [100]:
SELECT COUNT(DISTINCT e.patient)
       FROM Emergency e, Conditions c
       WHERE e.patient = c.patient AND
            (c.description_condition='Stroke');

42

## Patients with a Cardiac Arrest or Myocardial Infarction that Died 3 months after the Last Encounter

In [ ]:
SELECT COUNT(*)
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 90 AND
             (c.description_condition='Stroke');

SELECT DISTINCT p.first, p.last,
                to_char(p.birthdate,'dd-mm-yyyy') birth, to_char(p.deathdate,'dd-mm-yyyy') death,
                (p.deathdate-p.birthdate)/365 age,
                to_char(s.last_encounter,'dd-mm-yyyy') last_encounter,
                p.deathdate-s.last_encounter days
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 90 AND
             (c.description_condition='Stroke');

## Conditions in the Last Encounter with Code

In [ ]:
SELECT e.patient, c.code, c.description_condition
       FROM Emergency e, Conditions c
       WHERE e.patient = c.patient
       LIMIT 15;

## Patients that had Hypertension (code 59621000) in the Last Encounter

In [ ]:
SELECT COUNT(DISTINCT e.patient)
       FROM Emergency e, Conditions c
       WHERE e.patient = c.patient AND c.code='59621000';

## Patients with Hypertension that died 30 days after the Last Encounter

In [ ]:
SELECT COUNT(*)
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 30 AND
             c.code='59621000';

SELECT DISTINCT p.first, p.last,
                to_char(p.birthdate,'dd-mm-yyyy') birth, to_char(p.deathdate,'dd-mm-yyyy') death,
                (p.deathdate-p.birthdate)/365 age,
                to_char(s.last_encounter,'dd-mm-yyyy') last_encounter,
                p.deathdate-s.last_encounter days
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 30 AND
             c.code='59621000';

## Description of Conditions versus Encounters

In [ ]:
SELECT DISTINCT c.description_condition, e.description
       FROM conditions c, encounters e
       WHERE c.encounter = e.id AND encounterclass = 'emergency';

## Description of Conditions

In [ ]:
SELECT DISTINCT c.description_condition
       FROM conditions c, encounters e
       WHERE c.encounter = e.id AND encounterclass = 'emergency'
       ORDER BY c.description_condition;